# IMPORTING THE LIBRARIES 

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import statistics
import os

###############################################
from peakutils import indexes
from peakutils import baseline
from scipy.signal import find_peaks as fp
from scipy.signal import savgol_filter 
###############################################
from bokeh.plotting import figure , show
from pybaselines import whittaker as pl



from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

# DATA ARRANGEMENT

In the main directory we can see that there are 8 subfolders. 

Each subfolder have almost 12 spectras per sample , the idea behind that would be , instead of having just one spectra per sample , and to just rely on one information , its always better have to multiple measurements per samples , and then this could be used for building the Calibration Model

Instead of having 12 different csv per samples , its always good to have a single dataframe -> This new dataframe will have 1st column as wavelength , and 2nd -13th column as Intensities 

In [3]:
def load_data(folder_path):
    # List to store DataFrames for intensity columns
    intensity_dfs = []

    # List to store CSV file names
    csv_file_names = []

    # Loop through each file in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            
            # Read CSV file into a DataFrame
            df = pd.read_csv(file_path)
            
            # Extract intensity column and store it in the list
            intensity_dfs.append(df.iloc[:, 1])  # Assuming intensity is in the second column
            
            # Store CSV file name
            csv_file_names.append(os.path.splitext(file_name)[0])

    # Read the wavelength column from the first CSV file
    wavelength_df = pd.read_csv(os.path.join(folder_path, os.listdir(folder_path)[1]), usecols=[0])

    # Concatenate wavelength column with intensity columns
    result_df = pd.concat([wavelength_df] + intensity_dfs, axis=1)

    # Rename the columns with CSV file names
    column_names = ['wavelength'] + [f'{csv_file_names[i]}' for i in range(len(intensity_dfs))]

    result_df.columns = column_names
    # result_df.reset_index(drop=True, inplace=True)

    return result_df

In [4]:
def baseline_correction(df):
    """
    Perform baseline correction on the intensity columns of the input DataFrame and create a new DataFrame with corrected values.
    
    Parameters:
        df (DataFrame): Input DataFrame containing the wavelength and intensity columns.
        
    Returns:
        DataFrame: New DataFrame with baseline-corrected intensity columns and the same wavelength column as the input DataFrame.
    """
    # Copy the 'wavelength' column from the input DataFrame
    new_df = pd.DataFrame({'wavelength': df['wavelength']})
    
    # Perform baseline correction for each intensity column and add them to the new DataFrame
    for col in df.columns[1:]:  # Exclude the 'wavelength' column
        baseline, _ = pl.airpls(df[col],lam=0.1)
        corrected_values = df[col] - baseline
        new_df[col] = corrected_values
    
    return new_df

Loading the dataframe.

In [5]:
CR210LA_Raw_df = load_data('CR210LA')
###################################################
CR240LA_Raw_df = load_data('CR240LA')
###################################################
CR300LA_Raw_df = load_data('CR300LA')
###################################################
CR440Y_Raw_df = load_data('CR440Y')
###################################################
CR570CP_Raw_df = load_data('CR570CP')
###################################################
CR700Y_Raw_df = load_data('CR700Y')
###################################################
HR660Y_Raw_df = load_data('HR660Y')
# ###################################################
CR1000Y_Raw_df = load_data('CR1000Y')

In [6]:
CR210LA_BaselineCorrected_df = baseline_correction(CR210LA_Raw_df)

CR240LA_BaselineCorrected_df = baseline_correction(CR240LA_Raw_df)

CR300LA_BaselineCorrected_df = baseline_correction(CR300LA_Raw_df)

CR440Y_BaselineCorrected_df = baseline_correction(CR440Y_Raw_df)

CR570CP_BaselineCorrected_df = baseline_correction(CR570CP_Raw_df)

CR700Y_BaselineCorrected_df = baseline_correction(CR700Y_Raw_df)

HR660Y_BaselineCorrected_df = baseline_correction(HR660Y_Raw_df)

CR1000Y_BaselineCorrected_df = baseline_correction(CR1000Y_Raw_df)

# Peak Selection and Data Trimming

The dataframe  which we have is very big ,it could be trimmed now according to the wavelength , by adjusting two parameters "Wavelength_Min" , "Wavelength_Max"

In [7]:
Wavelength_Min = 203.8
Wavelength_Max = 204.3

CR210LA_Select_df = CR210LA_BaselineCorrected_df[(CR210LA_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (CR210LA_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]
CR240LA_Select_df = CR240LA_BaselineCorrected_df[(CR240LA_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (CR240LA_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]
CR300LA_Select_df = CR300LA_BaselineCorrected_df[(CR300LA_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (CR300LA_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]
CR440Y_Select_df = CR440Y_BaselineCorrected_df[(CR440Y_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (CR440Y_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]
CR570CP_Select_df = CR570CP_BaselineCorrected_df[(CR570CP_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (CR570CP_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]
CR700Y_Select_df = CR700Y_BaselineCorrected_df[(CR700Y_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (CR700Y_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]
HR660Y_Select_df = HR660Y_BaselineCorrected_df[(HR660Y_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (HR660Y_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]
CR1000Y_Select_df = CR1000Y_BaselineCorrected_df[(CR1000Y_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (CR1000Y_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]

CR210LA_Select_df


,wavelength,CR210LA_10,CR210LA_11,CR210LA_8,CR210LA_9,CR210LA_12,CR210LA_4,CR210LA_5,CR210LA_7,CR210LA_6,CR210LA_2,CR210LA_3,CR210LA_1
715,203.833333,51.163545,-1.966655,27.910181,19.186902,2.566485,27.856177,16.075348,16.462547,209.702518,35.708903,362.877921,101.160217
716,203.866667,0.000432,0.798688,20.754768,0.194112,-2.308181,28.806147,17.628269,16.734798,193.062654,28.066028,342.057976,76.826701
717,203.900000,-0.113082,53.126921,0.311372,32.265846,-12.964618,9.465514,2.508339,1.000007,161.983079,7.401812,303.529815,31.970919
718,203.933333,108.858638,203.496779,-0.149740,145.936273,10.411911,-0.093406,-0.164461,-0.051842,145.499405,-0.012634,278.192605,-0.026436
719,203.966667,368.293207,485.141887,109.653689,361.799042,210.288810,98.548707,103.847899,110.193477,232.751880,78.393405,366.475174,90.916063
720,204.000000,717.096321,837.894317,422.416328,639.898372,513.255639,406.212364,411.707398,430.137261,514.014626,315.969426,670.312976,416.920740
721,204.033333,1055.679009,1166.498863,875.577118,917.532448,803.205398,860.035191,860.926834,895.724977,930.736124,664.535512,1125.561998,910.477291
722,204.066667,1293.663118,1387.840289,1224.546367,1135.327823,980.445142,1205.245825,1201.839302,1252.431751,1249.875141,933.425324,1473.359373,1293.466242
723,204.100000,1377.190439,1466.826789,1230.763091,1247.169968,1010.179384,1193.316106,1190.958317,1252.019984,1244.060851,936.536826,1461.137903,1294.242234
724,204.133333,1261.489052,1380.248554,919.127093,1210.280758,873.649526,851.668101,855.410240,921.593381,936.726969,691.510505,1115.695405,937.936378


In [8]:
CR210LA_Select_Raw_df = CR210LA_Raw_df[(CR210LA_Raw_df['wavelength'] >= Wavelength_Min) & (CR210LA_Raw_df['wavelength'] <= Wavelength_Max)]
CR240LA_Select_Raw_df = CR240LA_Raw_df[(CR240LA_Raw_df['wavelength'] >= Wavelength_Min) & (CR240LA_Raw_df['wavelength'] <= Wavelength_Max)]
CR300LA_Select_Raw_df = CR300LA_Raw_df[(CR300LA_Raw_df['wavelength'] >= Wavelength_Min) & (CR300LA_Raw_df['wavelength'] <= Wavelength_Max)]
CR440Y_Select_Raw_df = CR440Y_Raw_df[(CR440Y_Raw_df['wavelength'] >= Wavelength_Min) & (CR440Y_Raw_df['wavelength'] <= Wavelength_Max)]
CR570CP_Select_Raw_df = CR570CP_Raw_df[(CR570CP_Raw_df['wavelength'] >= Wavelength_Min) & (CR570CP_Raw_df['wavelength'] <= Wavelength_Max)]
CR700Y_Select_Raw_df = CR700Y_Raw_df[(CR700Y_Raw_df['wavelength'] >= Wavelength_Min) & (CR700Y_Raw_df['wavelength'] <= Wavelength_Max)]
HR660Y_Select_Raw_df = HR660Y_Raw_df[(HR660Y_Raw_df['wavelength'] >= Wavelength_Min) & (HR660Y_Raw_df['wavelength'] <= Wavelength_Max)]
CR1000Y_Select_Raw_df = CR1000Y_Raw_df[(CR1000Y_Raw_df['wavelength'] >= Wavelength_Min) & (CR1000Y_Raw_df['wavelength'] <= Wavelength_Max)]

CR210LA_Select_Raw_df

,wavelength,CR210LA_10,CR210LA_11,CR210LA_8,CR210LA_9,CR210LA_12,CR210LA_4,CR210LA_5,CR210LA_7,CR210LA_6,CR210LA_2,CR210LA_3,CR210LA_1
715,203.833333,867.659119,974.856950,701.193542,992.970335,942.357671,626.974246,623.852278,648.214475,631.836555,593.787027,628.896138,710.133503
716,203.866667,817.327783,932.250251,683.449367,957.124225,893.644345,611.629193,607.352290,631.225787,617.774948,576.874441,612.454927,691.288018
717,203.900000,817.888732,943.248019,652.751758,972.540891,900.292943,579.040890,576.300907,599.034983,589.238982,549.822691,578.547737,652.293496
718,203.933333,927.379629,1056.208101,640.800893,1069.820502,1022.441858,559.652965,560.886494,582.759425,575.262918,538.581006,558.078608,626.541569
719,203.966667,1187.180123,1304.243229,739.399267,1269.622708,1301.721642,652.256055,656.415461,679.440723,664.985654,615.398220,651.481247,724.124598
720,204.000000,1536.197937,1627.064589,1041.276975,1532.057732,1666.105560,957.108068,959.473632,987.389137,948.679962,853.175936,960.695984,1056.911544
721,204.033333,1874.846132,1929.296083,1483.908376,1794.489760,2000.870316,1410.784031,1406.997650,1442.459099,1367.792973,1203.286338,1421.583682,1557.138739
722,204.066667,2112.748585,2127.701587,1822.739203,1997.611346,2207.706670,1757.949979,1748.744452,1790.034699,1689.282101,1474.615195,1775.286352,1946.433347
723,204.100000,2196.049117,2187.068404,1819.243895,2095.373965,2253.202839,1749.515152,1740.651399,1781.787300,1685.775172,1480.612574,1769.241673,1952.896644
724,204.133333,2079.977836,2084.065837,1498.357680,2045.065491,2119.983929,1412.339016,1409.268692,1444.729573,1380.704550,1238.471068,1430.252337,1601.406379


Lets plot a line plot ,to get a better picture 

In [9]:
Selected_df_Plot = figure(title = 'Selected Data Plot' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_1 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_2 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_3 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_4 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_5 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_6 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_7 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_8 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_9 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_10 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_11 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_12 , line_width = 2, color ="red" )

######################################################################################################################
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_1 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_2 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_3 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_4 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_5 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_6 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_7 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_8 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_9 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_10 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_11 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_12 , line_width = 2, color ="green" )

###############################################################################################################

Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_1 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_2 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_3 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_4 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_5 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_6 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_7 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_8 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_9 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_10 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_11, line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_12, line_width = 2, color ="pink")

######################################################################################################################

Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_1 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_2 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_3 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_4 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_5 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_6 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_7 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_8 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_9 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_10 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_11, line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_12, line_width = 2, color ="yellow")

##############################################################################################################

Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_1 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_2 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_3 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_4 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_5 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_6 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_7 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_8 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_9 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_10 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_11, line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_12, line_width = 2, color ="blue")

###############################################################################################################

Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_1 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_2 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_3 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_4 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_5 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_6 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_7 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_8 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_9 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_10 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_11, line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_12, line_width = 2, color ="orange")


####################################################################################################################

Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_1 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_2 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_3 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_4 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_5 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_6 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_7 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_8 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_9 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_10 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_11 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_12 , line_width = 2, color ="brown")


##################################################################################################################

Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_1 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_2 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_3 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_4 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_5 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_6 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_7 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_8 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_9 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_10 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_11 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_12 , line_width = 2, color ="black")

###################################################################################################################

Selected_df_Plot.width = 600
Selected_df_Plot.height = 500
show(Selected_df_Plot)

# Data Preprocessing of the Spectra

The above plot eventhough a spectra , is still a Raw Spectra , which still has lot of Artifects , before proceeding for the Univariate Calibration , its important to Pre Process the Raw Spectra accordingly. Various Pre Processing Techniques could be used here :- 

1) Baseline Correction - Very Very little  background radiation is still present in the spectra, which corresponds to the spectral baseline and imposes difficulties for quantitative elemental analysis.

In [10]:
Baseline_Correction_Plot = figure(title = 'Baseline Correction' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

Baseline_Correction_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_1 , line_width = 2, color ="green" )
Baseline_Correction_Plot.line(CR210LA_Select_Raw_df.wavelength,CR210LA_Select_Raw_df.CR210LA_1 , line_width =2 , color = "red")
Baseline_Correction_Plot.width = 600
Baseline_Correction_Plot.height = 500
show(Baseline_Correction_Plot)

2) Normalization - Its usually noticed that , for a measurement of a similar sample , the spectra obtained for them are quite different , but there could me many reasons for it , like laser energy fluctuations , material surface , even though is homegnous isnt same overall the surface

   For Instance , you could see the plot below , Though this are the plots from the same sample CR210LA , measured on 12 different Areas , its quite visible , for some  spectras , the peak heights or Intensities are not same.


In [11]:

Before_Normalization = figure(title = 'Before Normalization' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

Before_Normalization.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_1 , line_width = 2, color ="red" )
Before_Normalization.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_2 , line_width = 2, color ="red" )
Before_Normalization.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_3 , line_width = 2, color ="red" )
Before_Normalization.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_4 , line_width = 2, color ="red" )
Before_Normalization.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_5 , line_width = 2, color ="red" )
Before_Normalization.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_6 , line_width = 2, color ="red" )
Before_Normalization.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_7 , line_width = 2, color ="red" )
Before_Normalization.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_8 , line_width = 2, color ="red" )
Before_Normalization.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_9 , line_width = 2, color ="red" )
Before_Normalization.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_10 , line_width = 2, color ="red" )
Before_Normalization.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_11 , line_width = 2, color ="red" )
Before_Normalization.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_12 , line_width = 2, color ="red" )


Before_Normalization.width = 600
Before_Normalization.height = 500
show(Before_Normalization)

Now , its important the understand this chnages in Peaks , are common not just to a single peaks , but to the entire spectra. If theres a change in intensity of a peak , its not just for a single peak , but it applies to the entire spectrum proportionally. 

Normalization by peak of an Matrix Element i.e Fe 238.20 (237.9 ~ 238.7) , 263.10

Lets try to focus now on analysis for peak of Matrix Element

In [23]:
Matrix_Wavelength_Min = 237.9
Matrix_Wavelength_Max = 238.7

CR210LA_Matrix_df = CR210LA_BaselineCorrected_df[(CR210LA_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (CR210LA_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]
CR240LA_Matrix_df = CR240LA_BaselineCorrected_df[(CR240LA_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (CR240LA_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]
CR300LA_Matrix_df = CR300LA_BaselineCorrected_df[(CR300LA_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (CR300LA_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]
CR440Y_Matrix_df = CR440Y_BaselineCorrected_df[(CR440Y_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (CR440Y_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]
CR570CP_Matrix_df = CR570CP_BaselineCorrected_df[(CR570CP_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (CR570CP_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]
CR700Y_Matrix_df = CR700Y_BaselineCorrected_df[(CR700Y_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (CR700Y_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]
HR660Y_Matrix_df = HR660Y_BaselineCorrected_df[(HR660Y_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (HR660Y_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]
CR1000Y_Matrix_df = CR1000Y_BaselineCorrected_df[(CR1000Y_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (CR1000Y_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]

CR210LA_Matrix_df

,wavelength,CR210LA_10,CR210LA_11,CR210LA_8,CR210LA_9,CR210LA_12,CR210LA_4,CR210LA_5,CR210LA_7,CR210LA_6,CR210LA_2,CR210LA_3,CR210LA_1
1738,237.933333,5147.893025,2452.002677,2449.397919,1359.224258,3302.757332,4341.912700,2640.395098,2767.001164,2593.833167,1748.174177,3083.130902,3356.197268
1739,237.966667,4634.657139,2027.104612,2472.323285,1082.147909,2891.956457,4313.648860,2598.945002,2780.011055,2617.645638,1752.972368,3072.732633,3316.909646
1740,238.000000,4735.098910,1899.418961,2525.399370,906.473750,2914.358641,4310.365729,2584.940048,2826.526755,2664.963023,1735.155210,3072.015954,3232.368957
1741,238.033333,6096.779750,2828.783853,2909.212701,1386.259730,4056.689780,4706.026798,2969.915190,3244.484952,3050.593562,1929.476558,3467.596474,3485.038956
1742,238.066667,9334.668712,5586.936215,3950.073897,3097.090378,6996.723303,5821.724492,4106.775511,4347.769249,4117.654873,2709.356228,4636.279181,4518.615707
1743,238.100000,13973.695205,9780.438423,5987.874080,5832.192159,11294.702619,7950.639033,6338.590837,6437.564918,6224.209938,4521.419914,6949.689746,6809.121632
1744,238.133333,18142.810832,13523.652261,9052.886065,8382.851388,15079.687311,11085.282575,9678.949912,9516.525060,9406.351549,7480.999596,10433.893950,10448.748292
1745,238.166667,19994.548394,14957.545272,11780.547405,9548.277572,16511.032389,13865.324393,12658.111800,12243.459645,12249.658356,10208.557889,13559.031946,13799.893362
1746,238.200000,19239.062913,13907.040912,12417.456008,9123.595305,15427.963436,14553.341263,13392.834996,12902.597694,12935.327172,10907.618155,14361.460246,14742.518197
1747,238.233333,16708.007300,11409.521583,10209.032585,7620.994355,12879.223516,12391.950331,11064.487220,10752.504682,10677.578385,8793.354107,11981.588791,12334.551423


In [24]:
Selected_Matrix_Plot = figure(title = 'Selected Matrix Peak' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_1 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_2 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_3 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_4 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_5 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_6 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_7 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_8 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_9 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_10 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_11 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_12 , line_width = 2, color ="red" )

######################################################################################################################
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_1 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_2 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_3 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_4 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_5 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_6 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_7 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_8 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_9 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_10 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_11 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_12 , line_width = 2, color ="green" )

###############################################################################################################

Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_1 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_2 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_3 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_4 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_5 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_6 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_7 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_8 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_9 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_10 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_11, line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_12, line_width = 2, color ="pink")

######################################################################################################################

Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_1 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_2 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_3 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_4 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_5 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_6 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_7 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_8 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_9 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_10 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_11, line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_12, line_width = 2, color ="yellow")

##############################################################################################################

Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_1 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_2 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_3 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_4 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_5 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_6 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_7 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_8 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_9 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_10 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_11, line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_12, line_width = 2, color ="blue")

###############################################################################################################

Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_1 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_2 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_3 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_4 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_5 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_6 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_7 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_8 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_9 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_10 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_11, line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_12, line_width = 2, color ="orange")


####################################################################################################################

Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_1 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_2 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_3 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_4 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_5 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_6 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_7 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_8 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_9 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_10 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_11 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_12 , line_width = 2, color ="brown")


##################################################################################################################

Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_1 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_2 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_3 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_4 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_5 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_6 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_7 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_8 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_9 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_10 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_11 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_12 , line_width = 2, color ="black")

###################################################################################################################

Selected_Matrix_Plot.width = 600
Selected_Matrix_Plot.height = 500
show(Selected_Matrix_Plot)

In [ ]:

# def normalize_data(select_df, matrix_df):

#     # Iterate over each column (excluding the first column which is wavelength) in matrix_df
#     for sample_col in matrix_df.columns[1:]:
#         # Find peak intensity for the current sample column in matrix_df
#         peaks, _ = find_peaks(matrix_df[sample_col])
       
        
#         # Divide each intensity value in select_df by the peak intensity value
#         normalized_df[sample_col] = normalized_df[sample_col] / peak_intensity
    
#     return normalized_df



In [41]:
normalized_df = pd.DataFrame()

for sample_col in CR210LA_Matrix_df.columns[1:]:
        peaks, _ = fp(CR210LA_Matrix_df[sample_col], prominence=100 )
        return_intensities = CR210LA_Matrix_df[sample_col].iloc[peaks]
        correspondin_col = sample_col
        print(return_intensities)
        print(correspondin_col)
          
        

1745    19994.548394
Name: CR210LA_10, dtype: float64
CR210LA_10
1745    14957.545272
Name: CR210LA_11, dtype: float64
CR210LA_11
1746    12417.456008
Name: CR210LA_8, dtype: float64
CR210LA_8
1745    9548.277572
Name: CR210LA_9, dtype: float64
CR210LA_9
1745    16511.032389
Name: CR210LA_12, dtype: float64
CR210LA_12
1746    14553.341263
Name: CR210LA_4, dtype: float64
CR210LA_4
1746    13392.834996
Name: CR210LA_5, dtype: float64
CR210LA_5
1746    12902.597694
Name: CR210LA_7, dtype: float64
CR210LA_7
1746    12935.327172
Name: CR210LA_6, dtype: float64
CR210LA_6
1746    10907.618155
Name: CR210LA_2, dtype: float64
CR210LA_2
1746    14361.460246
Name: CR210LA_3, dtype: float64
CR210LA_3
1746    14742.518197
Name: CR210LA_1, dtype: float64
CR210LA_1
